In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma


In [3]:
from dotenv import load_dotenv
load_dotenv()


True

In [4]:
import pandas as pd
books= pd.read_csv("books_cleaned.csv")

In [5]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...


In [9]:
books["tagged_description"].to_csv("tagged_description.txt",
    index = False,
    header = False
)

In [13]:
raw_documents = TextLoader("tagged_description.txt", encoding="utf-8").load()

# Set chunk_size to a large, non-zero number
text_splitter = CharacterTextSplitter(chunk_size=6000, chunk_overlap=200, separator="\n")

documents = text_splitter.split_documents(raw_documents)

In [18]:
embeddings= GoogleGenerativeAIEmbeddings(model="models/embedding-001")
db_books= Chroma.from_documents(
    documents, embedding=embeddings,
)

In [20]:
query = "A book about kings"
docs = db_books.similarity_search(query, k = 10)
docs

[Document(id='a9ee8dc2-cdb9-4af4-9902-373e7b78dfef', metadata={'source': 'tagged_description.txt'}, page_content='"9780618181902 With their mother dead and their home burned, a thirteen-year-old boy and his little sister set out across Virginia in search of relatives during the final days of the Civil War."\n"9780618196258 In northern England in 1842, fourteen-year-old Lucas leads a lonely, monotonous existence in the house of his unpleasant guardian until the unexpected arrival of an unusual little girl presages a series of events that completely change his life."\n"9780618251452 The author of A Good Family offers poignant, entertaining account of how his and his son\'s mutual love for the music of the Beatles sparked a closer relationship, describing how they used the songs and exploits of the Fab Four to spark discussions of such topics as friendship, teamwork, art, sorrow, failure, and mortalitiy."\n"9780618254118 A new historian of Mary Queen of Scots draws on new sources to shatt

In [22]:

def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10,
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k = 50)

    books_list = []

    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)]



In [23]:
retrieve_semantic_recommendations("A book about sea")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
18,9780006498865,0006498868,The Mad Ship,Robin Hobb,Fantasy fiction,http://books.google.com/books/content?id=2iWez...,Fantasy master Robin Hobb delivers the stunnin...,2000.0,4.24,906.0,36101.0,The Mad Ship,9780006498865 Fantasy master Robin Hobb delive...
76,9780007204465,0007204469,Tropic of Cancer,Henry Miller,Fiction,http://books.google.com/books/content?id=ProgR...,"Miller's groundbreaking first novel, banned in...",2005.0,3.69,336.0,709.0,Tropic of Cancer,9780007204465 Miller's groundbreaking first no...
441,9780066620107,0066620104,What Would Machiavelli Do?,Stanley Bing,Humor,http://books.google.com/books/content?id=W9jVl...,What Would Machiavelli Do? He would feast on o...,2002.0,3.46,176.0,434.0,What Would Machiavelli Do?: The Ends Justify t...,9780066620107 What Would Machiavelli Do? He wo...
459,9780099271499,0099271494,Blood and Gold,Anne Rice,Fiction,http://books.google.com/books/content?id=kq8A8...,Here is the gorgeous and sinister story of Mar...,2002.0,3.89,752.0,27476.0,Blood and Gold: The Vampire Marius,9780099271499 Here is the gorgeous and siniste...
467,9780099285557,009928555X,Set this House on Fire,William Styron,Americans,http://books.google.com/books/content?id=TDaEP...,The day after Peter Leverett met his old frien...,2001.0,3.70,567.0,551.0,Set this House on Fire,9780099285557 The day after Peter Leverett met...
482,9780099428169,0099428164,In the Pond,Ha Jin,China,http://books.google.com/books/content?id=bkmry...,Ha Jin has been compared to the late Issac Bab...,2002.0,3.71,160.0,1212.0,In the Pond,9780099428169 Ha Jin has been compared to the ...
533,9780099582014,0099582015,Sabbath's Theater,Philip Roth,American fiction,http://books.google.com/books/content?id=lm070...,"Once a scandalously inventive puppeteer, Micke...",1996.0,3.85,451.0,122.0,Sabbath's Theater,9780099582014 Once a scandalously inventive pu...
849,9780143036692,0143036696,The Mermaid Chair,Sue Monk Kidd,Fiction,http://books.google.com/books/content?id=3pRoQ...,Jessie Sullivan is summoned home to tiny Egret...,2006.0,3.12,368.0,66723.0,The Mermaid Chair,9780143036692 Jessie Sullivan is summoned home...
869,9780143039495,0143039490,The Tale of Genji,Murasaki Shikibu,Fiction,http://books.google.com/books/content?id=ikbgg...,The first translation of the Japanese masterpi...,2006.0,3.71,319.0,148.0,The Tale of Genji: Abridged,9780143039495 The first translation of the Jap...
929,9780156286510,0156286513,E. M. Forster,Philip Nicholas Furbank,Biography & Autobiography,http://books.google.com/books/content?id=KrvbA...,"P. N. Furbank's 1978 two-volume portrait, comb...",1981.0,4.11,648.0,133.0,E. M. Forster: A Life,9780156286510 P. N. Furbank's 1978 two-volume ...
